In [ ]:
#Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import os, sys, gc, time, warnings, pickle, psutil, random
import matplotlib.pyplot as plt
import statsmodels.api as sm
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
import seaborn as sns
from IPython.display import Image, display
warnings.filterwarnings('ignore')

In [ ]:
#Helper Functions to clean data can't be shown due to confidentiality

In [ ]:
#Preaparing data for propensity score matching

In [ ]:
 #define function to generate dataret for latter matching use
def psm(df):
    median1 = df.test_center_resident_ratio.median()
    median2 = df.test_center_take_test_ratio.median()
    median3 = df.prep_course_ratio.median()
    median4 = df.prep_test_ratio.median()
    median5 = df.prep_resource_ratio.median()

    df['test_center_resident_ratio'] = np.where(df['test_center_resident_ratio']<=median1, 0,
    df['test_center_resident_ratio'])
    df['test_center_resident_ratio'] = np.where(df['test_center_resident_ratio'] >median1, 1,
    df['test_center_resident_ratio'])
    df['test_center_take_test_ratio'] = np.where(df['test_center_take_test_ratio'] <= median2, 0,
    df['test_center_take_test_ratio'])
    df['test_center_take_test_ratio'] = np.where(df['test_center_take_test_ratio'] > median2, 1,
    df['test_center_take_test_ratio'])
    df['prep_course_ratio'] = np.where(df['prep_course_ratio'] <= median3, 0,df['prep_course_ratio'])
    df['prep_course_ratio'] = np.where(df['prep_course_ratio'] > median3, 1,df['prep_course_ratio'])
    df['prep_test_ratio'] = np.where(df['prep_test_ratio'] <= median4, 0,df['prep_test_ratio'])
    df['prep_test_ratio'] = np.where(df['prep_test_ratio'] > median4, 1,df['prep_test_ratio'])
    df['prep_resource_ratio'] = np.where(df['prep_resource_ratio'] <= median5, 0,df['prep_resource_ratio'])
    df['prep_resource_ratio'] = np.where(df['prep_resource_ratio'] > median5, 1,df['prep_resource_ratio'])
    X = df[['total_population', 'edu_less_highshcool', 'total_edu_greater_highschool',
    'total_income_less_45k', 'total_income_greater_45k', 'race_white','race_asian',
    'race_african_american','female_population','employment_rate']]
    y = df['prep_resource_ratio']
    return df

In [ ]:
f = pd.read_csv('candidate_based.csv')
df = psm(df)

In [ ]:
df.to_csv('matching.csv', index=False)

In [ ]:
#matching was done in R

In [ ]:
#importing results from Matching done in R

In [ ]:
# read the test resource dataset after PSM 
df1 = pd.read_csv("match_test_center_take_test_ratio.csv")
df2 = pd.read_csv("match_test_center_resident_ratio.csv")
df3 = pd.read_csv("match_prep_test_ratio.csv")
df4 = pd.read_csv("match_prep_course_ratio.csv")

In [ ]:
#running Linear regressions

In [ ]:
# do ols regression on all the data

In [ ]:
#regress all test passing rate on teh test center utilization
y1 = df1['All_test_passing_rate']
x1 = df1[['test_center_take_test_ratio']]
x1 = sm.add_constant(x1)
model1 = sm.OLS(y1,x1)
fitmodel1 = model1.fit()

In [ ]:
#Below, we regress the all test passing rate on the test center capacity. 
#We mentioned the calculation of test center capacity in matching.
y2 = df2['All_test_passing_rate']
x2 = df2[['test_center_resident_ratio']]
x2 = sm.add_constant(x2)
model2 = sm.OLS(y2,x2)
fitmodel2 = model2.fit()
print("\n")
print(fitmodel2.summary())
print("\n\n\n")

In [ ]:
#Below, we regress the all test passing rate on prep course. We mentioned the calculation of prep course in matching.

In [ ]:
y4 = df4['All_test_passing_rate']
x4 = df4[['prep_course_ratio']]
x4 = sm.add_constant(x4)
model4 = sm.OLS(y4,x4)
fitmodel4 = model4.fit()
print("\n")
print(fitmodel4.summary())
print("\n\n\n")

In [ ]:
# regress the all test passing rate on prep test. We mentioned the calculation of prep test in matching
y3 = df3['All_test_passing_rate']
x3 = df3[['prep_test_ratio']]
x3 = sm.add_constant(x3)
model3 = sm.OLS(y3,x3)
fitmodel3 = model3.fit()
print("\n")
print(fitmodel3.summary())
print("\n\n\n"

In [ ]:
#K-means clustering

In [ ]:
 # dataset used for clustering
df_cluster = df_city.iloc[:, 3:]
# put all of the KPI we set into new dataset for clustering use
df_cluster = df_city[['state','county','city','exam_math_median_score','exam_reasoning_median_score',
 'exam_science_median_score','exam_social_studies_median_score',
'All_test_passing_rate','math_pass_rate','reasoning_pass_rate',
 'science_pass_rate','social_studies_pass_rate']].iloc[:, 3:]
# do min-max normalization to transform all of the features included into same scale
df_standard = MinMaxScaler().fit_transform(df_cluster)

In [ ]:
#selecting number of clusters

In [ ]:
distortions = []
# use different number of clusters from 1-9 to generate different result for below error-rate checking
K = range(1,10)
for k in K:
 kmeanModel = KMeans(n_clusters=k)
 kmeanModel.fit(df_standard)
 distortions.append(kmeanModel.inertia_

In [ ]:
 # plot out the Elbow Curve to find the optimal number of clusters to use
# in this case, we use 2 as our cluster number
plt.plot(K, distortions, 'bx-')
plt.xlabel(k)
plt.ylabel('Distortion')
ax = plt.gca()
ax.set_xticks(range(1,k+1))
plt.title('The Elbow Method showing the optimal k')
plt.show()

In [ ]:
# generate 2 clusters using our post-normalization city-level KPI dataset
kmeans = KMeans(n_clusters = 2, random_state = 1)
kmeans.fit(df_standard)

In [ ]:
# map the clustering result, aka the label back to each city
df_kmeans = df_city.copy()
df_kmeans['label'] = kmeans.labels_

In [ ]:
 # dataset stats overview
df_kmeans.groupby('label').mean().T

In [ ]:
 data size for each cluster
df_kmeans.groupby('label').size()

In [ ]:
#second level regression - only on worst performing cities

In [ ]:
# test resource treatment
df_test_resource_treatment = [df3, df4]
# test resource treatment
test_resource_treatment = ["prep_test_ratio","prep_course_ratio"]
df_list_bad = []

In [ ]:
# prepare bad cluster matching data
# path
bad_match_bad_test_resource = '../bad_match_bad_test_resource/'
# define bad match bed test resource treatment function
def bad_bad_test_resource(df_test_resource_treatment):
    for i, df in enumerate(df_test_resource_treatment):
 
    df = df_test_resource_treatment[i]
  
    df = df[df['label'] == 1]
 
    df = df.sort_values(["subclass",test_resource_treatment[i]])
 
    df = df.groupby("subclass").filter(lambda x:len(x)==2)
 
    df_list_bad.append(df)
    df.to_csv(bad_match_bad_test_resource + test_resource_treatment[i] + "_bad.csv",index=False)

In [ ]:
bad_bad_test_resource(df_test_resource_treatment)

In [ ]:
def regression_bad(df_list_bad):
    i = 0
    for kpi in test_resource_treatment:
        y = df_list_bad[i]['All_test_passing_rate']
        x = df_list_bad[i][[kpi]]
        x = sm.add_constant(x)
        model = sm.OLS(y,x)
        fitmodel = model.fit()
        print(kpi + " regression")
        print("\n")
        print(fitmodel.summary())
        print("\n\n\n")
        i += 1

In [ ]:
regression(df_list_bad)

In [ ]:
#study aides helpfulness regression

In [ ]:
# read the prep course dataset after PSM 
dfb1 = pd.read_csv("STUDY_HELPFUL_ADULT_EDUCATION_CLASS.csv")
dfb2 = pd.read_csv("STUDY_HELPFUL_ADULT_EDUCATION_TEACHER.csv")
dfb3 = pd.read_csv("STUDY_HELPFUL_AUDIO_STUDY_MATERIALS.csv")
dfb4 = pd.read_csv("STUDY_HELPFUL_BOOKS_PRINTED_STUDY_MATERIAL.csv")
dfb5 = pd.read_csv("STUDY_HELPFUL_exam_READY.csv")
dfb6 = pd.read_csv("STUDY_HELPFUL_MATERIALS_MOBILE_APP.csv")
dfb7 = pd.read_csv("STUDY_HELPFUL_ONLINE_COURSE_VIDEO_STUDY_MATERIALS.csv")
dfb8 = pd.read_csv("STUDY_HELPFUL_OTHER.csv")
dfb9 = pd.read_csv("STUDY_HELPFUL_SOCIAL_NETWORKING_WEBSITE.csv")
dfb10 = pd.read_csv("STUDY_HELPFUL_TV_STUDY_PROGRAM.csv")
dfb11 = pd.read_csv("STUDY_LOCATION_TEST_PREPARATION_CENTER.csv"

In [ ]:
# prep course treatment list
prep_course_treatment =['STUDY_HELPFUL_ADULT_EDUCATION_CLASS','STUDY_HELPFUL_ADULT_EDUCATION_TEACHER',
 'STUDY_HELPFUL_AUDIO_STUDY_MATERIALS','STUDY_HELPFUL_BOOKS_PRINTED_STUDY_MATERIAL',
'STUDY_HELPFUL_MATERIALS_MOBILE_APP',
'STUDY_HELPFUL_ONLINE_COURSE_VIDEO_STUDY_MATERIALS',
'STUDY_HELPFUL_SOCIAL_NETWORKING_WEBSITE','STUDY_HELPFUL_TV_STUDY_PROGRAM',
'STUDY_LOCATION_TEST_PREPARATION_CENTER']
# file dataframe list
df_prep_resource_list= [dfb1, dfb2, dfb3, dfb4,dfb6, dfb7, dfb9, dfb10, dfb11]
# path
bad_prep_course = '../bad_prep_course/'
df_list_help = []
# define bad match bad prep course treatment function
def bad_bad_prep_course(df_prep_resource_list,prep_course_treatment):
    for i, df in enumerate(df_prep_resource_list):
        df = df_prep_resource_list[i]
        df = df[df['label'] == 1]
        df = df.sort_values(["subclass",prep_course_treatment[i]])
        df = df.groupby("subclass").filter(lambda x:len(x)==2)
        df.to_csv(bad_prep_course + prep_course_treatment[i] + "_bad.csv",index=False)
        df_list_help.append(df)

In [ ]:
bad_bad_prep_course(df_prep_resource_list,prep_course_treatment)

In [ ]:
def regression_help(df_list_help):
    i = 0
    for kpi in prep_course_treatment:
        y = df_list_help[i]['All_test_passing_rate']
        x = df_list_help[i][kpi]
        x = sm.add_constant(x)
        model = sm.OLS(y,x)
        fitmodel = model.fit()
        print(kpi + " regression")
        print("\n")
        print(fitmodel.summary())
        print("\n\n\n)
        i+=1

In [ ]:
 regression_help(df_list_help)